In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.integrate import solve_ivp

In [2]:
m1,m2 = np.random.uniform(0.5,5.0,size=2)

In [3]:
m1

np.float64(1.4780714028007538)

In [4]:
m2

np.float64(0.712475060395064)

In [5]:
x1 = np.random.uniform(-4,4,size=2)

In [6]:
x2 = np.random.uniform(-10,10,size=2)

In [7]:
x2

array([0.66528943, 1.96302026])

In [8]:
x1.shape

(2,)

In [9]:
def two_body_ode(t, y, G, m1, m2):
    x1 = y[0:2]
    v1 = y[2:4]
    x2 = y[4:6]
    v2 = y[6:8]

    r12 = x2 - x1
    r = np.linalg.norm(r12)
    if r == 0:
        return np.zeros(8)

    a1 = G * m2 * r12 / r**3
    a2 = -G * m1 * r12 / r**3

    dydt = np.zeros(8)
    dydt[0:2] = v1
    dydt[2:4] = a1
    dydt[4:6] = v2
    dydt[6:8] = a2

    return dydt

def generate_sample(G=1.0, t_max=50.0):
    # Random masses and initial conditions
    m1, m2 = np.random.uniform(0.1, 10.0, size=2)
    x1 = np.random.uniform(-10, 10, size=2)
    x2 = np.random.uniform(-10, 10, size=2)
    v1 = np.random.uniform(-5, 5, size=2)
    v2 = np.random.uniform(-5, 5, size=2)

    y0 = np.concatenate([x1, v1, x2, v2])

    # Choose a random time t_eval < t_max
    t_eval = [np.random.uniform(0.5, t_max)]  # Avoid t=0 which is just initial state

    # Solve ODE
    sol = solve_ivp(two_body_ode, [0, t_max], y0, args=(G, m1, m2), t_eval=t_eval)

    if not sol.success:
        return None  # Skip failed cases

    final_state = sol.y[:, -1]

    return {
        "m1": m1, "m2": m2,
        "x1_0": x1.tolist(), "x2_0": x2.tolist(),
        "v1_0": v1.tolist(), "v2_0": v2.tolist(),
        "t": t_eval[0],
        "x1_t": final_state[0:2].tolist(),
        "x2_t": final_state[4:6].tolist(),
    }


In [10]:
from tqdm import tqdm

In [11]:
data = []
for _ in tqdm(range(11000)):  # Number of samples
    sample = generate_sample()
    if sample:
        data.append(sample)

df = pd.DataFrame(data)
df.to_csv("two_body_dataset.csv", index=False)

100%|██████████| 11000/11000 [01:36<00:00, 113.69it/s]
